In [165]:
import json, re, string

In [25]:
with open('data/dimlex.json') as j:
    dimlex_raw = json.load(j)

In [175]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [244]:
conn_dict = {}
for entry in dimlex_raw['entry']:
    
    word_regex = '(^| )(' + entry['word']
    if '...' in word_regex:
        word_regex = re.sub('\s?\.{3}\s?',' [^.?!]+ ',word_regex)
    for orth in entry['orths']['orth']:
        orth_variant = []
        for part in orth['part']:
            orth_variant.append(part['t'])
        orth_variant = ' '.join(orth_variant)
        word_regex += '|' + orth_variant
        # print(orth_variant)
    word_regex += ')( |\.|\?|!|$)'
    if word_regex not in conn_dict:
        conn_dict[word_regex] = entry['ambiguity']['non_conn']
    else: 
        print(dimlex_raw['entry'].index(entry), word_regex, entry['ambiguity']['non_conn'], 'already in dict')
    # if entry['word'] not in conn_dict:
    #    non_conn_dict[entry['word']] = entry['ambiguity']['non_conn']
    # else:
    #    print(entry['word'], 'already in dict')

In [132]:
conn_dict2 = {}
for entry in dimlex_raw['entry']:
    orths_list = []
    for orth in entry['orths']['orth']:
        orth_variant = []
        for part in orth['part']:
            orth_variant.append(part['t'])
        orth_variant = ' '.join(orth_variant)
        if orth_variant not in conn_dict2:
            conn_dict2[orth_variant] = entry['ambiguity']['non_conn']
        else: 
            print(dimlex_raw['entry'].index(entry), orth_variant, entry['ambiguity']['non_conn'], 'already in dict')
            if conn_dict2[orth_variant] == {'t': 0}:
                conn_dict2[orth_variant] = entry['ambiguity']['non_conn']
                print('replaced previous entry')
    # if entry['word'] not in conn_dict:
    #    non_conn_dict[entry['word']] = entry['ambiguity']['non_conn']
    # else:
    #    print(entry['word'], 'already in dict')

87 drum {'freq': 19, 'anno_N': 25, 't': 1} already in dict
87 Drum {'freq': 19, 'anno_N': 25, 't': 1} already in dict
190 so dass {'t': 0} already in dict
190 so daß {'t': 0} already in dict
263 zuletzt {'freq': 25, 'anno_N': 25, 't': 1} already in dict


In [252]:
conn_dict

{'(^| )(aber|aber|Aber)( |\\.|\\?|!|$)': {'freq': 3, 'anno_N': 21, 't': 1},
 '(^| )(abgesehen davon|abgesehen davon|Abgesehen davon)( |\\.|\\?|!|$)': {'freq': 47,
  'anno_N': 73,
  't': 1},
 '(^| )(abgesehen davon, dass|abgesehen davon , dass|Abgesehen davon , dass|abgesehen davon , daß|Abgesehen davon , daß)( |\\.|\\?|!|$)': {'t': 0},
 '(^| )(alldieweil|alldieweil|Alldieweil)( |\\.|\\?|!|$)': {'t': 0},
 '(^| )(allein|allein|Allein)( |\\.|\\?|!|$)': {'freq': 49,
  'anno_N': 50,
  't': 1},
 '(^| )(allenfalls|allenfalls|Allenfalls)( |\\.|\\?|!|$)': {'freq': 51,
  'anno_N': 51,
  't': 1},
 '(^| )(allerdings|allerdings|Allerdings)( |\\.|\\?|!|$)': {'t': 0},
 '(^| )(als|als|Als)( |\\.|\\?|!|$)': {'freq': 54, 'anno_N': 61, 't': 1},
 '(^| )(als ob|als ob|Als ob)( |\\.|\\?|!|$)': {'freq': 18,
  'anno_N': 30,
  't': 1},
 '(^| )(als wenn|als wenn|Als wenn)( |\\.|\\?|!|$)': {'freq': 22,
  'anno_N': 37,
  't': 1},
 '(^| )(alsdann|alsdann|Alsdann)( |\\.|\\?|!|$)': {'freq': 1,
  'anno_N': 18,
  't':

In [88]:
html_string="<!DOCTYPE html>\n<html>\n<head>\n<title>Page Title</title>\n</head>\n<body>\n<p>\n" # start of paragraph
html = open('html/output.html', mode='w', encoding='utf8')        
            # <h1>This is a Heading</h1> \
with open("data/coronaimpfung.txt") as txt: # https://www.dw.com/de/coronaimpfung-die-m%C3%A4r-von-der-unfruchtbarkeit/a-58733733
    for line in txt:
        splitted_line = line.split() # TODO: write proper tokenizer, add casing variants, punctuation
        if len(splitted_line) == 0: html_string+="</p>\n<p>" 
        for word in splitted_line:
            if word in non_conn_dict:
                #print(word)
                if non_conn_dict[word]['t'] == 0:
                    html_string+=" <font color=\"green\">{word}</font> ".format(word=word)
                else:
                    html_string+=" <font color=\"red\">{word}</font> ".format(word=word)
            else:
                html_string+=" {word} ".format(word=word)
        html_string+="\n"
    html_string+="</p>"

In [283]:
html_string="<!DOCTYPE html>\n<html>\n<head>\n<title>Page Title</title>\n</head>\n<body>\n<p>\n" # start of paragraph
html = open('html/output.html', mode='w', encoding='utf8')        
            # <h1>This is a Heading</h1> \
with open("data/coronaimpfung.txt") as txt: # https://www.dw.com/de/coronaimpfung-die-m%C3%A4r-von-der-unfruchtbarkeit/a-58733733
    for line in txt:
        if line.strip() == '': html_string+="</p>\n<p>" 
        html_line = line
        for conn_reg in conn_dict:
            if re.search(conn_reg, html_line):
                # print(conn_reg, html_line, conn_dict[conn_reg]['t'])
                if conn_dict[conn_reg]['t'] == 0:
                    html_line = re.sub(conn_reg, r' <font color="green">\g<2></font> ', html_line)
                else: 
                    html_line = re.sub(conn_reg, r' <font color="red">\g<2></font> ', html_line)
            # if match: print(line, match)
        html_string+=html_line
        html_string+="\n"
html_string+="</body>\n</html>"

In [284]:
html.write(html_string)
html.write('<style>\n \
    * {font-family: monospace;} \
            </style>")')
html.close()

In [278]:
html_string

'<!DOCTYPE html>\n<html>\n<head>\n<title>Page Title</title>\n</head>\n<body>\n<p>\nCoronaimpfung - die Mär von der Unfruchtbarkeit\n\n</p>\n<p>\n\nDas Gerücht ist nicht neu, hält sich <font color="red">aber</font> hartnäckig: Die Impfstoffe gegen COVID-19 können unfruchtbar machen – Männer <font color="red">wie</font> Frauen. Ist diese Sorge begründet? <font color="red">Und</font> <font color="red">wie</font> gefährlich ist <font color="red">dann</font> die COVID-Infektion selbst?\n\n"Die Impfung gegen das SARS-2 Virus macht unfruchtbar!" Solche <font color="red">und</font> ähnliche Aussagen  kursieren <font color="red">seit</font> Monaten in den sozialen Medien. \n\n</p>\n<p>\n\n <font color="red">Zunächst</font> hieß es, vor allem Frauen müssten um ihre Fortpflanzungsfähigkeit bangen – eine biologische Erklärung <font color="red">dafür</font> wurde <font color="red">auch</font> mitgeliefert: Die <font color="red">durch</font> den Impfstoff gebildeten Antikörper könnten nicht <font co

In [250]:
s = "hält sich aber hartnäckig aber."

In [251]:
re.sub('(^| )(aber|aber|Aber)( |\.|\?|!|$)', r' <font color=\"green\">\g<2></font> ', s)

'hält sich <font color=\\"green\\">aber</font> hartnäckig <font color=\\"green\\">aber</font> '

In [224]:
re.search('[\s^](aber|aber|Aber)[\\s\\.?!\\$]', s)